<a href="https://colab.research.google.com/github/TonmoyTalukder/Rank-Your-Summaries-Enhancing-Bengali-Text-Summarization-via-Ranking-based-Approach/blob/main/Summary%20Generation%20with%20Pre-trained%20Models/scibert_scivocab_uncased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset Loading

## Dataset I

In [ ]:
#install dataset
!pip install datasets

In [ ]:
from datasets import load_dataset
from pprint import pprint

In [ ]:
traindf = load_dataset('csebuetnlp/xlsum', 'bengali', split='train')

In [ ]:
testdf = load_dataset('csebuetnlp/xlsum', 'bengali', split='test')

In [ ]:
valdf = load_dataset('csebuetnlp/xlsum', 'bengali', split='validation')

In [ ]:
print("Dataset length: ", len(traindf))
print("\n First item dataset[0]: ")
pprint(traindf[1])

In [ ]:
pprint(len(traindf[0]['text']))
pprint(type(traindf[0]['text']))

9356
<class 'str'>


In [ ]:
import pandas as pd

df = pd.DataFrame(traindf)

df.head()

,id,url,title,summary,text
0,news-54690291,https://www.bbc.com/bengali/news-54690291,দুর্গাপূজার সময়ে যেভাবে শোক পালন করেন 'মহিষাস...,হিন্দু বাঙালীরা যে সময়ে তাদের সবথেকে বড় উৎসব...,দুর্গাপুজায় মহিষাসুর বধ্যে মধ্য দিয়ে অশুভর ও...
1,news-50797621,https://www.bbc.com/bengali/news-50797621,রাশিয়ায় ক্ষমতার ২০ বছর যেভাবে কেটেছে ভ্লাদিম...,ভ্লাদিমির পুতিন তাঁর ক্ষমতায় থাকার ২০ বছর পূর...,গত ২০ বছরে তিনি রাশিয়ার প্রেসিডেন্ট এবং প্রধা...
2,news-46678346,https://www.bbc.com/bengali/news-46678346,সংসদ নির্বাচন: বরিশালে ছয়টি আসন কিন্তু সবার দ...,বাংলাদেশের দক্ষিণাঞ্চলীয় জেলা বরিশাল এখন তুমু...,বরিশাল সদরে চলছে নির্বাচনী প্রচার প্রচারণা। যদ...
3,news-51860832,https://www.bbc.com/bengali/news-51860832,সর্বকালের সর্বশ্রেষ্ঠ বাঙালি: বিবিসি বাংলার জর...,দু'হাজার চার সালে বিবিসি বাংলা একটি 'শ্রোতা জর...,রবীন্দ্রনাথ ঠাকুর রবীন্দ্রনাথ ঠাকুর বাঙালির কা...
4,56310420,https://www.bbc.com/bengali/56310420,৭ই মার্চের ভাষণ: ৫০ বছর আগে রেসকোর্স ময়দানে উ...,'ভাষণ শুরু আগে মাথার উপর দিয়ে বিমান আর হেলিকপ...,আর কুমিল্লা থেকে বাস ভাড়া করে অনেকের সাথে নিজ...


In [ ]:
testdf = pd.DataFrame(testdf)

testdf

In [ ]:
valdf = pd.DataFrame(valdf)

valdf

In [ ]:
df.head(10)

In [ ]:
df['text'][2]

In [ ]:
rows = df.shape[0]
rows

8102

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8102 entries, 0 to 8101
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       8102 non-null   object
 1   url      8102 non-null   object
 2   title    8102 non-null   object
 3   summary  8102 non-null   object
 4   text     8102 non-null   object
dtypes: object(5)
memory usage: 316.6+ KB


In [ ]:
df.describe()

,id,url,title,summary,text
count,8102,8102,8102,8102,8102
unique,8102,8102,8101,8098,8102
top,news-54690291,https://www.bbc.com/bengali/news-54690291,বাংলাদেশে শিশু ধর্ষণের ঘটনা বাড়ছে কেন?,বাংলাদেশে পরবর্তী প্রধান নির্বাচন কমিশনার হিসে...,দুর্গাপুজায় মহিষাসুর বধ্যে মধ্য দিয়ে অশুভর ও...
freq,1,1,2,2,1


## Dataset II

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/SCDataset/Final/text_summarization.csv')

df.head()

,category,summary,text
0,technology,অ্যাপসে মিলবে ঢাকাসহ তিন জেলা আদালতের তথ্য,"ঢাকা মহানগর ও ঢাকা জেলা আদালত, কিশোরগঞ্জ ও রাঙ..."
1,bangladesh,বিজ্ঞান ও প্রকৌশলে মার্কিন সর্বোচ্চ সম্মাননা...,যুক্তরাষ্ট্রে বিজ্ঞান ও প্রকৌশলে পেশা শুরুর প...
2,bangladesh,বিকল্প শিশুখাদ্য গ্রহণে শিশুর মৃত্যু হলে শাস্ত...,বিকল্প শিশুখাদ্য ও বাণিজ্যিকভাবে উত্পাদিত শিশু...
3,bangladesh,ট্রেনে কাটা পড়ে সাবেক সিভিল সার্জনের মৃত্যু,বগুড়ার আদমদীঘির সান্তাহারে গতকাল শনিবার ট্রেন...
4,bangladesh,যাত্রাবাড়ীতে চুলা জ্বালাতে গিয়ে দুই কর্মচারী ...,যাত্রাবাড়ীর একটি রেস্তোরাঁয় গতকাল বুধবার ভোর...


In [ ]:
df.head(10)

,category,summary,text
0,technology,অ্যাপসে মিলবে ঢাকাসহ তিন জেলা আদালতের তথ্য,"ঢাকা মহানগর ও ঢাকা জেলা আদালত, কিশোরগঞ্জ ও রাঙ..."
1,bangladesh,বিজ্ঞান ও প্রকৌশলে মার্কিন সর্বোচ্চ সম্মাননা...,যুক্তরাষ্ট্রে বিজ্ঞান ও প্রকৌশলে পেশা শুরুর প...
2,bangladesh,বিকল্প শিশুখাদ্য গ্রহণে শিশুর মৃত্যু হলে শাস্ত...,বিকল্প শিশুখাদ্য ও বাণিজ্যিকভাবে উত্পাদিত শিশু...
3,bangladesh,ট্রেনে কাটা পড়ে সাবেক সিভিল সার্জনের মৃত্যু,বগুড়ার আদমদীঘির সান্তাহারে গতকাল শনিবার ট্রেন...
4,bangladesh,যাত্রাবাড়ীতে চুলা জ্বালাতে গিয়ে দুই কর্মচারী ...,যাত্রাবাড়ীর একটি রেস্তোরাঁয় গতকাল বুধবার ভোর...
5,bangladesh,‘বাল্যবিবাহমুক্ত’ জেলা ঘোষণা চলছে নিয়ম না মেনেই,পর্যাপ্ত তথ্য-উপাত্ত হাতে নিয়েই কোনো জেলা বা...
6,sports,পাকিস্তানি ক্রিকেটারদের দুটির বেশি টি-টোয়েন্টি...,বাংলাদেশ প্রিমিয়ার লিগে গতবার প্রতি ম্যাচে পাঁ...
7,bangladesh,চাটখিল উপজেলা ভাইস চেয়ারম্যান সন্ত্রাসী হামলার...,লক্ষ্মীপুরের পোদ্দার বাজারে সন্ত্রাসী হামলার শ...
8,bangladesh,ব্যবসায়ীদের চাপে জরিমানা কমাতে বাধ্য হলেন বিচারক,নিষিদ্ধ পলিথিন ব্যবসায় জড়িত থাকায় এক ব্যবসা...
9,bangladesh,নাইকো দুর্নীতি মামলায় আপিল করলেন খালেদা জিয়া,নাইকো দুর্নীতি মামলায় হাইকোর্টের দেওয়া রায়ের ব...


In [ ]:
rows = df.shape[0]
rows

80331

# Install and Imports

In [ ]:
!pip install sentencepiece
!pip install transformers
!pip install git+https://github.com/csebuetnlp/normalizer

import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer


from transformers import XLMProphetNetForConditionalGeneration, XLMProphetNetTokenizer


from transformers import MBartForConditionalGeneration, AutoModelForSeq2SeqLM
from transformers import AlbertTokenizer, AutoTokenizer

In [ ]:
!pip install bert-extractive-summarizer

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

### Functions

In [ ]:
def cleanText(text2clean):
    empty_string = ""

    space = chr(32)
    flag  = 0

    for i in range(len(text2clean)):
        if text2clean[i] != "'" and text2clean[i] != "\n":
            if text2clean[i] == space and flag == 0:
                flag = 1
                empty_string = empty_string + text2clean[i]
            elif text2clean[i] != space:
                empty_string = empty_string + text2clean[i]
                flag = 0
    return empty_string

# Model Loading 6: allenai/scibert_scivocab_uncased

### Model Setup

In [ ]:
from transformers import *

# Load model, model config and tokenizer via Transformers
custom_config = AutoConfig.from_pretrained('allenai/scibert_scivocab_uncased')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
custom_model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased', config=custom_config)

from summarizer import Summarizer

In [ ]:
model6 = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)

In [ ]:
# df['text'][2]

In [ ]:
# testText = 'বাঁহাতি স্পিনে আফগানিস্তানের ব্যাটিং লাইন-আপ প্রায় একাই ধসিয়ে দিলেন বাঁহাতি স্পিনার মাহফুজুর রহমান। রান তাড়ায় বাংলাদেশ অনূর্ধ্ব-১৯ দলের জয়ের লক্ষ্যটা থাকল দেড় শরও নিচে। এরপর সাবলীল ব্যাটিংয়ে ব্যাটসম্যানরা দলকে লক্ষ্যে পৌঁছে দিলেন সহজেই। আবুধাবিতে আজ আফগানিস্তানের বিপক্ষে ৬ উইকেটের জয়ে ত্রিদেশীয় অনূর্ধ্ব-১৯ টুর্নামেন্টে চ্যাম্পিয়ন হয়েছে বাংলাদেশ যুব দল। প্রথমে ব্যাট করতে নামা আফগানিস্তান মাহফুজের স্পিন ঘূর্ণিতে ৩৭ ওভারে ১৪৩ রানে অলআউট হয়। রান তাড়ায় ২৩.২ ওভারে ৪ উইকেট হারিয়ে লক্ষ্যে পৌঁছে যায় বাংলাদেশ। বাংলাদেশ, আফগানিস্তান ও শ্রীলঙ্কার যুব দলকে নিয়ে আয়োজিত ত্রিদেশীয় টুর্নামেন্টটি শুরু হয় গত ১৮ মার্চ। ডাবল-রাউন্ড পদ্ধতির টুর্নামেন্টে প্রথম দুই ম্যাচেই হেরে গিয়েছিল বাংলাদেশ। এর পর টানা দুই জয় তুলে ফাইনালে জায়গা করে আহরার আমিনের দল। আফগানিস্তান ফাইনালে ওঠে চার ম্যাচের তিনটিতে জিতে। আবুধাবির টোলেরেন্স স্টেডিয়ামে অনুষ্ঠিত ফাইনালে অবশ্য আফগানদের ওড়ার সুযোগই দেয়নি বাংলাদেশ। ৫ ওভারে ২৮ রান তোলা উদ্বোধনী জুটি ভাঙেন বাঁহাতি পেসার মারুফ মৃধা। মাহফুজের স্পিন-ঘূর্ণির আঘাত শুরু ওয়াফিউল্লাহ তারাখিলকে বোল্ড করে। ৫ চারে ৩৬ বলে ২৭ রান করে যান তারাখিল। আফগানিস্তানে ইনিংসে এর চেয়ে বেশি রান করেছেন শুধু হারুন খান। নয় নম্বরে আউট হওয়া এই ব্যাটসম্যান ৬২ বলে খেলে যান ৬৫ রানের ইনিংস। তার আগেই আফগান ব্যাটিং লাইন-আপে ধস নামান মাহফুজ। ১০ ওভার হাত ঘুরিয়ে ১ মেডেনসহ ২৯ রানে তুলে নেন ৬ উইকেট। টুর্নামেন্টে সব মিলিয়ে ১০ উইকেট নিলেন ১৭ বছর বয়সী এই বাঁহাতি। আরেক বাঁহাতি রাফিউজ্জামান নেন ৩৩ রানে ২ উইকেট। রান তাড়ায় চতুর্থ ওভারেই ওপেনার আশিকুজ্জামানকে হারায় বাংলাদেশ। তার আগেই ডানহাতি এ উইকেটকিপার-ব্যাটসম্যান তিন চারে ১৩ বলে ১৭ রান করেন। তিনে নামা জিশান আলমও ছিলেন আক্রমণাত্মক- ৭টি বাউন্ডারিতে করেন ১৯ বলে ৩৫ রান। তৃতীয় উইকেট জুটিতে চৌধুরী মোহাম্মদ রিজওয়ান (৪৩) ও আরিফুল ইসলাম (২২) মিলে ৫৩ রান যোগ করলে সহজ জয়ের দিকে এগিয়ে যায় বাংলাদেশ। একপর্যায়ে দুজনই আউট হলেও জয় এসেছে সহজেই।'

testText = 'সারা জীবন যে মানুষটার সঙ্গে কাটাতে হবে, সবার আগে তো তাকে জানা চাই। তবে শেষ এক মাসের ধকল যেন আর নেওয়া যায় না। উপভোগ তো দূরের কথা, কোনোমতে বিয়েটা শেষ করতে পারলেই যেন বাঁচেন। আর যাঁদের ‘অ্যারেঞ্জ ম্যারেজ’, তাঁদের অবস্থাটা তাহলে বুঝুন। আয়োজন ছোট হোক বা বড়, বর-কনের ওপর চাপ পড়েই। বিয়ের পর মধুচন্দ্রিমার আবির্ভাব তাই আশীর্বাদের মতো। কিন্তু টয়ার ভাষায়, ‌‘ওটাকে ছোট্ট হানিমুন বলতে পারেন। এবার হয়তো আমরা দেশের বাইরে কোথাও যাব, সেটাকেই হানিমুন ট্রিপ বলব।’ দুজনেই নিজেদের মতো করে প্রকাশ করতে চান নিজেকে। ‘কোনো কারণে একজন রেগে গেলে, আরেকজন চুপ থাকুন। আরাম করা, একে অপরকে সময় দেওয়া - অনেকটাই যেন বাধ্যতামূলক কাজ এই সময়। তিন-চার দিনের এই বিশেষ সময়টা নানা জায়গায় ছোটাছুটি করে নষ্ট না করাই ভালো।’ স্বামী-স্ত্রী দুজনেই যখন ঘুরতে পছন্দ করেন, তখন অনেকটাই সহজ হয়ে যায় ভ্রমণ। সার্ফিং, স্কুবা ডাইভিং,জঙ্গলের মধ্যে হেঁটে বেড়ানো - কত কী যে করেছেন। পোশাকের রং হিসেবে বেছে নিতে বললেন লাল, নীল, হলুদ আর সাদা। জঙ্গলে বা চা-বাগানে বেশি ভালো লাগবে সাদা রঙের পোশাক। কারণ, এতে একে অপরকে খুঁজে নিতে পারবেন যথার্থভাবে। এ জন্য দায়ী মুলত দুটি কারণ, কম ঘুম আর রোদে থাকা।'

article_text = normalize(testText)

summary = model6(article_text, max_length=76)

print(summary)

### Run dataset over the Model

In [ ]:
sciDF = pd.read_csv("/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased_Combined_v.csv")
sciDF= sciDF.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
newRows = sciDF.shape[0]
sciDF

,title,text,givenSummary,scibert_scivocab_uncased
0,দুর্গাপূজার সময়ে যেভাবে শোক পালন করেন 'মহিষাস...,দুর্গাপুজায় মহিষাসুর বধ্যে মধ্য দিয়ে অশুভর ও...,হিন্দু বাঙালীরা যে সময়ে তাদের সবথেকে বড় উৎসব...,দুর্গাপুজায় মহিষাসুর বধ্যে মধ্য দিয়ে অশুভর ও...
1,রাশিয়ায় ক্ষমতার ২০ বছর যেভাবে কেটেছে ভ্লাদিম...,গত ২০ বছরে তিনি রাশিয়ার প্রেসিডেন্ট এবং প্রধা...,ভ্লাদিমির পুতিন তাঁর ক্ষমতায় থাকার ২০ বছর পূর...,গত ২০ বছরে তিনি রাশিয়ার প্রেসিডেন্ট এবং প্রধা...
2,সংসদ নির্বাচন: বরিশালে ছয়টি আসন কিন্তু সবার দ...,বরিশাল সদরে চলছে নির্বাচনী প্রচার প্রচারণা। যদ...,বাংলাদেশের দক্ষিণাঞ্চলীয় জেলা বরিশাল এখন তুমু...,যদিও প্রায় সব আসনেই প্রচার প্রচারণা বেশি চোখে...
3,সর্বকালের সর্বশ্রেষ্ঠ বাঙালি: বিবিসি বাংলার জর...,রবীন্দ্রনাথ ঠাকুর রবীন্দ্রনাথ ঠাকুর বাঙালির কা...,দু'হাজার চার সালে বিবিসি বাংলা একটি 'শ্রোতা জর...,বাংলা সাহিত্যের তিনি একজন উজ্জ্বল নক্ষত্র এবং ...
4,৭ই মার্চের ভাষণ: ৫০ বছর আগে রেসকোর্স ময়দানে উ...,আর কুমিল্লা থেকে বাস ভাড়া করে অনেকের সাথে নিজ...,'ভাষণ শুরু আগে মাথার উপর দিয়ে বিমান আর হেলিকপ...,আর কুমিল্লা থেকে বাস ভাড়া করে অনেকের সাথে নিজ...
...,...,...,...,...
8097,ট্রাম্পের বোধশক্তি যাচাইয়ের টেস্টে পাশ করবেন তো?,ডোনাল্ড ট্রাম্প 'ডোনাল্ড ট্রাম্পের বোধশক্তি ঠি...,আরো পড়ুন:,NaN
8098,মঙ্গল গ্রহে প্রাণের অস্তিত্ব সম্পর্কে যা জানা ...,মঙ্গলগ্রহে তরল পানি থাকার প্রমাণ খুঁজে পেয়েছে...,আরো পড়ুন:,"তাদের বিশ্বাস, এটি একটি হৃদ এবং সেটি রাডার ব্য..."
8099,কীভাবে বুঝবেন আপনার সন্তান কিশোর গ্যাং কালচারে...,বরগুনায় রিফাত শরীফ নামের এক যুবককে প্রকাশ্যে ...,ঘটনা এক:,বরগুনায় রিফাত শরীফ নামের এক যুবককে প্রকাশ্যে ...
8100,সৌদি আরব - ইরান দ্বন্দ্ব: কে কার বন্ধু?,স্যেদি যুবরাজ মোহাম্মদ বিন সালমান ও ইরানের প্র...,সৌদি আরব,স্যেদি যুবরাজ মোহাম্মদ বিন সালমান ও ইরানের প্র...


In [ ]:
 custom = [378, 544, 1043, 2879, 2922, 4082, 6086, 6850, 7168, 8097]
 custom[0]

378

In [ ]:
sciDF['text'][custom[0]]

In [ ]:
art = "সুবীর ভৌমিক, বিবিসি সিকিমে রাজা ও রানীর সাথে সত্যজিত রায়৻ (ছবি: সিকিম টাইমস্‌) প্রয়াত সত্যজিত রায়ের ছেলে সন্দীপ রায় সরকারি এই সিদ্ধান্তের কথা নিশ্চিত করেছেন৻ উৎফুল্ল সন্দীপ রায় বলেন, তিনি আশা করছেন দ্রুত দর্শকরা তার বাবার তৈরী এই প্রামাণ্য চলচ্চিত্রটি দেখতে পাবেন৻ চলচ্চিত্রটি তৈরির সময়কার স্মৃতি হাতড়ে সন্দীপ রায় বলেন, শ্যুটিংয়ের সময় ক্যামেরা এবং জিনিসপত্র নিয়ে সিকিমে পাহাড়ে ওঠানামার কথা তার চোখের সামনে ভাসছে৻ সিকিমের ওপর তৈরি এই প্রামাণ্য চলচ্চিত্রটি তৈরি শেষ হয়েছিলো ১৯৭৩ সালের দিকে৻ সিকিম সেসময় একটি স্বাধীন রাজতন্ত্র ছিলো৻ তাঁর দেশে পর্যটকদের আকর্ষণ করতে তৎকালীন রাজা পালডেন থনডুপ নামগিয়্যাল সত্যজিত রায়কে এই প্রামাণ্য চলচ্চিত্রটি তৈরির দায়িত্ব দিয়েছিলেন৻ সন্দীপ রায় জানান, রাজা নামগিয়্যালের আমেরিকান স্ত্রী হোপ কুকের সাথে তার বাবার বন্ধুত্ব ছিলো, এবং সেই সূত্রেই রাজার সাথে যোগাযোগ৻ চলচ্চিত্র নির্মাতা হিসেবে খ্যাতি পাওয়ার পর সত্যজিত রায় যে দুটো প্রামাণ্য চলচ্চিত্র তৈরি করেছিলেন, ‘সিকিম‘ ছিলো তারই একটি৻ অন্যটি ছিলো কবি রবীন্দ্রনাথ ঠাকুরের উপর৻ শট নিয়ে ক্ষুব্ধ রাজা সিকিমের রাজা নামগিয়্যাল ও রানী কুক৻ (ছবি: সিকিম টাইমস্‌) সিকিম তৈরি হওয়ার পর এর একটি শট নিয়ে রাজা এবং তার আমেরিকান স্ত্রী তীব্র আপত্তি তুলেছিলেন৻ রাজধানী গ্যাংটকে রাজকীয় একটি ভোজসভার পর ফেলে দেয়া খাবার নিয়ে প্রাসাদের বাইরে দরিদ্র মানুষের কাড়াকাড়ির এক দৃশ্যে ক্ষুব্ধ রাজা শটটি বাদ দিতে বলেন সত্যজিত রায়কে৻ সন্দীপ রায় জানান, তার বাবা শটটি বাদ দিয়ে চলচ্চিত্রটি চূড়ান্ত করার পরপরই সিকিমের রাজনৈতিক পট আমূল বদলে যায়৻ ১৯৭৫ সালে সিকিম ভারতের সাথে যুক্ত হয়৻ সেসময় যে কোন ধরণের বিতর্ক এড়াতে ভারত সরকার সত্যজিত রায়ের এই প্রামাণ্য চলচ্চিত্রটির ওপর নিষেধজ্ঞা আরোপ করে৻ প্রায় চার দশক বাদে নিষেধাজ্ঞা ওঠার পর কবে কোথায় চলচ্চিত্রটি প্রথমবারের মত দেখা যাবে তা এখনো পরিস্কার নয়৻ ভারত সরকারের চলচ্চিত্র বিভাগের একজন কর্মকর্তা জানিয়েছেন তাদের নিজস্ব নেটওয়ার্কের মাধ্যমে এটি দেখানোর চেষ্টা হতে পারে৻ জানা গেছে, সিকিমের একটি প্রিন্ট আমেরিকাতে এবং আরেকটি ব্রিটিশ ফিল্ম ইনিস্টিউটে রক্ষিত আছে৻"

In [ ]:
brt = "- কীভাবে বুঝবেন শিশুর অতিরিক্ত খাবার দরকার? - শুরুতে শিশুদের কী ধরণের খাবার দেয়া উচিত? - শিশুকে কি ডিম খাওয়ানো যাবে? - শিশুর খাবার কীভাবে তৈরি করবেন? - খিচুড়ি কীভাবে খাওয়াবেন? - জুস, আইসক্রিম কখন দেবেন? - শিশুকে কতটুকু খাবার, কখন দেবেন? - কী কী খাবার এড়িয়ে যাবেন? এসব প্রশ্নের উত্তর জানতে ভিডিওটি পুরোটি দেখুন।"

In [ ]:
# Create empty DataFrame
genDf = pd.DataFrame(columns = ["title", "text", "givenSummary", "scibert_scivocab_uncased"])

custom = [378, 544, 1043, 2879, 2922, 4082, 6086, 6850, 7168, 8097]

# Load over the dataset
for j in range(len(custom)):
    # if i < 1000:
    # if i > 999 and i < 2000:
    # if i > 1999 and i < 3000:
    # if i > 2999 and i < 4000:
    # if i > 3999 and i < 5000:
    # if i > 4999 and i < 6000:
    # if i > 5999 and i < 7000:
    # if i > 6999 and i < 8000:
    if i > 7999:
    i = custom[j]
    article_text = normalize(myNorm(sciDF['text'][i]))

    summary = model6(article_text, max_length=400)

    # sciDF.loc[i] = [sciDF['title'][i], sciDF['text'][i], sciDF['summary'][i], summary]
    sciDF.at[[i],'scibert_scivocab_uncased']=summary

    print(i, '  ', summary)
    # break


#genDf.to_csv('/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased/scibert_scivocab_uncased_v1000.csv')
# genDf.to_csv('/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased/scibert_scivocab_uncased_v2000.csv')
# genDf.to_csv('/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased/scibert_scivocab_uncased_v3000.csv')
# genDf.to_csv('/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased/scibert_scivocab_uncased_v4000.csv')
# genDf.to_csv('/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased/scibert_scivocab_uncased_v5000.csv')
# genDf.to_csv('/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased/scibert_scivocab_uncased_v6000.csv')
# genDf.to_csv('/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased/scibert_scivocab_uncased_v7000.csv')
# genDf.to_csv('/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased/scibert_scivocab_uncased_v8000.csv')
genDf.to_csv('/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased/scibert_scivocab_uncased_v8KPlus.csv')

## Concat Splitted Dataset

In [ ]:
df0 = pd.read_csv("/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased/scibert_scivocab_uncased_v1000.csv")
df1 = pd.read_csv("/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased/scibert_scivocab_uncased_v2000.csv")
df2 = pd.read_csv("/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased/scibert_scivocab_uncased_v3000.csv")
df3 = pd.read_csv("/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased/scibert_scivocab_uncased_v4000.csv")
df4 = pd.read_csv("/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased/scibert_scivocab_uncased_v5000.csv")
df5 = pd.read_csv("/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased/scibert_scivocab_uncased_v6000.csv")
df6 = pd.read_csv("/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased/scibert_scivocab_uncased_v7000.csv")
df7 = pd.read_csv("/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased/scibert_scivocab_uncased_v8000.csv")
df8 = pd.read_csv("/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased/scibert_scivocab_uncased_v8KPlus.csv")


frames = [df0, df1, df2, df3, df4, df5, df6, df7, df8]
genDfCombined = pd.concat(frames)
genDfCombined.to_csv('/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased_Combined_v.csv', index=False)
sciDF.to_csv('/content/drive/MyDrive/SCDataset/scibert_scivocab_uncased_Combined_v.csv', index=False)

In [ ]:
sciDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8102 entries, 0 to 8101
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   title                     8102 non-null   object
 1   text                      8102 non-null   object
 2   givenSummary              8102 non-null   object
 3   scibert_scivocab_uncased  8102 non-null   object
dtypes: object(4)
memory usage: 253.3+ KB


In [ ]:
sciDF[sciDF['scibert_scivocab_uncased'].isnull()].index.tolist()